### Melon 100 Chart 
* 100곡의 노래의 제목, Song ID추출 (정규표현식 사용)
* Song ID로 상세페이지를 100번 요청해서 노래 상세정보 추출
* 100곡의 상세정보를 json file에 저장한다.
* json file을 Pandas의 DataFrame 객체에 저장해서 표 데이터를 만들기
* 표 데이터를 MariaDB에 저장하기

#### 1.100곡의 노래의 제목, Song ID추출 (정규표현식 사용)

In [25]:
import requests
from bs4 import BeautifulSoup
import re

url = 'https://www.melon.com/chart/index.htm'
req_header = {
    'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36'
}

# 요청을 보내기
res = requests.get(url, headers=req_header)
print(res.status_code)
# 응답결과가 정상이면
if res.ok:
    # 응답 객체에 포함된 text를 추출
    html = res.text
    # 추출한 text를 파싱하기 위한 BeautifulSoup 객체생성
    soup = BeautifulSoup(html, 'html.parser')
    # <div id='tb_list'> div 태그 선택 ,1개
    print(len(soup.select('div#tb_list')))
    # <div id='tb_list'> 하위에 있는 tr 태그 선택 ,101개
    print(len(soup.select('div#tb_list tr')))
    # <div id='tb_list'> 하위에 있는 tr 하위에 <div class='wrap_song_info'> , 200개
    print(len(soup.select('div#tb_list tr .wrap_song_info')))
    print(len(soup.select('div.wrap_song_info')))
    # <div id='tb_list'> 하위에 있는 tr 하위에 <div class='wrap_song_info'> 하위에 a
    print(len(soup.select('div#tb_list tr .wrap_song_info a'))) #458
    print(len(soup.select('div.wrap_song_info a'))) #458
    # <div id='tb_list'> 하위에 있는 tr 하위에 <div class='wrap_song_info'> 
    # 하위에 <a href='playSong'> 
    print(len(soup.select('div#tb_list tr .wrap_song_info a[href*=\'playSong\']')))
    
    atag_list = soup.select("div#tb_list tr .wrap_song_info a[href*='playSong']")
    print('-------------')
    print(type(atag_list), type(atag_list[2]), atag_list[2].text)
    
    # 100곡 노래 저장할 list
    song_list = []
    for idx, atag in enumerate(atag_list, 1):
        # 1곡 노래 저장할 dict
        song_dict = {}
        
        # a tag 의 content
        title = atag.text
        # a tag의 href 속성 값
        href = atag['href']
        # 정규표현식 패턴으로 숫자를 검색
        matched = re.search(r'(\d+)\);', href)
        #print(matched, matched.group(0), matched.group(1))
        song_id = matched.group(1)
        #print(idx, title, song_id)
        song_url = f'https://www.melon.com/song/detail.htm?songId={song_id}'
        
        #추출한 song title과 url를 song_dict에 저장
        song_dict['title'] = title
        song_dict['url'] = song_url
        #1개의 노래정보를 담고 있는 song_dict를 song_list에 추가
        song_list.append(song_dict)

print(len(song_list))        
song_list[0:3]

200
1
101
200
200
458
458
100
-------------
<class 'bs4.element.ResultSet'> <class 'bs4.element.Tag'> Dynamite
100


[{'title': 'VVS (Feat. JUSTHIS) (Prod. GroovyRoom)',
  'url': 'https://www.melon.com/song/detail.htm?songId=33077590'},
 {'title': '밤하늘의 별을(2020)',
  'url': 'https://www.melon.com/song/detail.htm?songId=33061995'},
 {'title': 'Dynamite',
  'url': 'https://www.melon.com/song/detail.htm?songId=32872978'}]

### 2. Song ID로 상세페이지를 100번 요청해서 노래 상세정보 추출

In [26]:
import requests
from bs4 import BeautifulSoup
import re

req_header = {
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.75 Safari/537.36'
}

#노래 100곡의 상세정보를 저장할 list
song_detail_list = []

for song in song_list:
    #노래 1곡의 상세정보를 저장할 dict
    song_detail_dict = {}
    
    #print(song['title'], song['link'])
    song_url = song['url']
    res = requests.get(song_url, headers=req_header)
    print(res.status_code)
    if res.ok:
        html = res.text
        soup = BeautifulSoup(html,'html.parser')
        #노래제목
        song_detail_dict['곡명'] = song['title']
        #가수
        song_detail_dict['가수'] = \
            soup.select('a[href*="goArtistDetail"] span')[0].text
        #앨범
        song_detail_dict['앨범'] = soup.select('div.meta dd')[0].text
        #발매일
        song_detail_dict['발매일'] = soup.select('div.meta dd')[1].text
        #장르
        song_detail_dict['장르'] = soup.select('div.meta dd')[2].text
        #가사
        lyric = soup.select('div#d_video_summary')[0].text
        #정규표현식 Pattern 생성
        regexp = re.compile(r'[\n\r\t]')     
        #Pattern과 매칭되는 문자열로 empty string으로 치환해라
        song_detail_dict['가사'] = regexp.sub('',lyric.strip())
        
        #각각의 Song정보를 담고 있는 dict를 list에 저장
        song_detail_list.append(song_detail_dict)
#         print('-----')
#         print(song_detail_dict)    

print(len(song_detail_list))
print(song_detail_list[0:3])

200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
100
[{'곡명': 'VVS (Feat. JUSTHIS) (Prod. GroovyRoom)', '가수': '미란이 (Mirani)', '앨범': '쇼미더머니 9 Episode 1', '발매일': '2020.11.21', '장르': '랩/힙합', '가사': "보여줘야겠어내가 망할 거랬던 놈들에게도내가 잘될 거라 했던 너에게도다 할게 최선갈아끼워 다 새 거My life's shining like a VVS VVS내 삶은 빛나 VVS VVS보란 듯 나와 TV에Now you see me babe내 삶은 빛나 VVS VVSSkrt-skrtHey new water new vv난 알바 째고 무대 위Yeah go get it go get it가사 위 가난이 빛나지안 가 무한리필살아봐야겠어 내 빌딩 Yeah개 같던 세상의뒤통수 치러 왔지더 크게 Callin’ ma name모두 날 보고 놀래미란이가 TV에떼버려 Tag사 새롭게 Yeah yeah타고 비행Skrr skrr 난 올라가Skrr skrr 난 빛이 나내가 뭐라 했어 Mom꺼내겠다고 포차맨 밑바닥의 소녀엄마의 술병이 날 만들어허기져 이를 꽉 물어Chit chat bout me 덤벼 겁쟁이 너VVS on ma neck 꿈

#### 3. 100곡의 상세정보를 json file에 저장한다.

In [27]:
import json

with open('data/songs.json', 'w', encoding='utf-8') as file:
    json.dump(song_detail_list, file)